In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
email_data = pd.read_csv('H:\\Naive Byes\\sms_raw_NB.csv',encoding='ISO-8859-1')

In [16]:
email_data

,type,text
0,ham,hope having good week just checking
1,ham,give back thanks
2,ham,also doing only have
3,spam,complimentary star ibiza holiday cash needs yo...
4,spam,okmail dear dave this your final notice collec...
...,...,...
5554,ham,great role model giving much really wish each ...
5555,ham,awesome remember last time somebody high first...
5556,spam,your prize will another customer polo suite lo...
5557,spam,jsco energy high know where channel leadership...


In [6]:
stop_words = []
with open('H://study//NLP//Dataset/stopwords_en.txt') as sw:
    stop_words = sw.read()

In [8]:
stop_words.split('\n')

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

In [9]:
def cleaning_txt(i):
    i = re.sub("[^A-Za-z" "]+", " ", i).lower()
    i = re.sub("[0-9" "]+", " ", i)
    w = []
    for word in i.split(" "):
        if len(word) > 3:
            w.append(word)
    return (" ".join(w))

In [10]:
#testing func
cleaning_txt("Hope you are having a good week. Just checking in")

'hope having good week just checking'

In [11]:
cleaning_txt("hope i can understand your feelings 123121. 123 hi how .. are you?")

'hope understand your feelings'

In [13]:
email_data.text = email_data.text.apply(cleaning_txt)

In [14]:
email_data.text

0                     hope having good week just checking
1                                        give back thanks
2                                    also doing only have
3       complimentary star ibiza holiday cash needs yo...
4       okmail dear dave this your final notice collec...
                              ...                        
5554    great role model giving much really wish each ...
5555    awesome remember last time somebody high first...
5556    your prize will another customer polo suite lo...
5557    jsco energy high know where channel leadership...
5558                          shall call dear having food
Name: text, Length: 5559, dtype: object

In [17]:
# removing empty rows
email_data = email_data.loc[email_data.text != " "]

In [18]:
email_data

,type,text
0,ham,hope having good week just checking
1,ham,give back thanks
2,ham,also doing only have
3,spam,complimentary star ibiza holiday cash needs yo...
4,spam,okmail dear dave this your final notice collec...
...,...,...
5554,ham,great role model giving much really wish each ...
5555,ham,awesome remember last time somebody high first...
5556,spam,your prize will another customer polo suite lo...
5557,spam,jsco energy high know where channel leadership...


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
email_train,email_test = train_test_split(email_data,test_size=0.2)

In [50]:
def split_into_words(i):
    return [word for word in i.split(" ")]

In [53]:
email_bag_of_words = CountVectorizer(analyzer= lambda i: [word for word in i.split(" ")]).fit(email_data.text)
# Defining the preparation of email texts into word count matrix format - Bag of Words

In [54]:
#bow for all msg
all_email_mtx = email_bag_of_words.transform(email_data.text)

In [55]:
#for tarining msgs
train_email_mtx = email_bag_of_words.transform(email_train.text)

In [56]:
 #For testing messages
test_emails_mtx = email_bag_of_words.transform(email_test.text)

In [57]:
test_emails_mtx

<1112x6661 sparse matrix of type '<class 'numpy.int64'>'
	with 8347 stored elements in Compressed Sparse Row format>

In [60]:
# Learning Term weighting and normalizing on entire emails
#tfidf_transform = TfidfVectorizer().fit(all_email_mtx)

In [61]:
# Learning Term weighting and normalizing on entire emails
tfidf_transformer = TfidfTransformer().fit(all_email_mtx)

In [62]:
#preparing tf-idf for train
train_tfidf = tfidf_transformer.transform(train_email_mtx)

In [63]:
train_tfidf.shape

(4447, 6661)

In [65]:
#tfidf for test
test_tfidf = tfidf_transformer.transform(test_emails_mtx)

In [67]:
test_tfidf.shape

(1112, 6661)

In [68]:
from sklearn.naive_bayes import MultinomialNB as MB

In [69]:
classifier_mb = MB()
classifier_mb.fit(train_tfidf,email_train.type)

MultinomialNB()

In [70]:
#evaluate on test data
test_pred_m = classifier_mb.predict(test_tfidf)

In [71]:
acc_test_m = np.mean(test_pred_m == email_test.type)

In [72]:
acc_test_m

0.9586330935251799

In [73]:
from sklearn.metrics import accuracy_score

In [74]:
accuracy_score(test_pred_m,email_test.type)

0.9586330935251799

In [75]:
pd.crosstab(test_pred_m, email_test.type)

type,ham,spam
row_0,,
ham,948,46
spam,0,118


In [76]:
# Training Data accuracy
train_pred_m = classifier_mb.predict(train_tfidf)
accuracy_train_m = np.mean(train_pred_m == email_train.type)
accuracy_train_m

0.9680683606926017

In [77]:
# Multinomial Naive Bayes changing default alpha for laplace smoothing
# if alpha = 0 then no smoothing is applied and the default alpha parameter is 1
# the smoothing process mainly solves the emergence of zero probability problem in the dataset.

In [ ]:
classifier_mb_lap = MB(alpha=3)

In [78]:
classifier_mb_lap.fit(train_tfidf,email_train.type)

MultinomialNB(alpha=3)

In [79]:
# Evaluation on Test Data after applying laplace
test_pred_lap = classifier_mb_lap.predict(test_tfidf)



In [80]:
accuracy_test_lap = np.mean(test_pred_lap == email_test.type)


In [81]:
accuracy_test_lap

0.9010791366906474

In [82]:
accuracy_score(test_pred_lap, email_test.type)

pd.crosstab(test_pred_lap, email_test.type)

type,ham,spam
row_0,,
ham,948,110
spam,0,54


In [84]:
pd.crosstab(test_pred_m, email_test.type)

type,ham,spam
row_0,,
ham,948,46
spam,0,118


In [83]:
# Training Data accuracy
train_pred_lap = classifier_mb_lap.predict(train_tfidf)
accuracy_train_lap = np.mean(train_pred_lap == email_train.type)
accuracy_train_lap

0.9138745221497638